In [ ]:
import io
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
conn= sqlite3.connect('tp1.db')
cursor= conn.cursor()

In [ ]:
f= io.open('despesas_publicas_tp1.sql', 'r')
sql= f.read()
cursor.executescript(sql)

In [ ]:
df= pd.read_sql_query("SELECT * FROM ORGAO ORDER BY QTD_ORGAOS_SUBORDINADOS DESC", conn)
df

In [ ]:
df = pd.read_sql_query("SELECT P.NOME_PROGRAMA_GOVERNO, SUM(D.VALOR_DESPESA) AS VALOR_DESPESA " + 
                       "FROM DESPESA_PUBLICA AS D " +
                       "INNER JOIN PROGRAMA_GOVERNO AS P " +
                       "ON D.COD_PROGRAMA_GOVERNO = P.COD_PROGRAMA_GOVERNO " + 
                       "GROUP BY P.NOME_PROGRAMA_GOVERNO " +
                       "HAVING SUM(VALOR_DESPESA) > 0 " +
                       "ORDER BY VALOR_DESPESA DESC", conn)
df

In [ ]:
df = pd.read_sql_query("SELECT D.MES_DESPESA, SUM(D.VALOR_DESPESA) AS VALOR_DESPESA " + 
                       "FROM DESPESA_PUBLICA AS D " +
                       "INNER JOIN DETALHAMENTO_FUNCAO AS DF " +
                       "ON D.COD_DETALHAMENTO = DF.COD_DETALHAMENTO " +
                       "INNER JOIN FUNCAO AS F " +
                       "ON DF.COD_FUNCAO = F.COD_FUNCAO " +
                       "WHERE F.NOME_FUNCAO = 'Educação' " +
                       "GROUP BY D.MES_DESPESA " +
                       "ORDER BY D.MES_DESPESA", conn)
df

In [ ]:
plt.bar(df['MES_DESPESA'].tolist(), df['VALOR_DESPESA'].tolist())

plt.xlabel("Mês/2018")
plt.ylabel("Total Despesa Mensal")
plt.show()

# 1. Liste todos os códigos e nomes de funções da subfunção que possui o maior número de funções.


In [ ]:
df= pd.read_sql_query("SELECT cod_funcao, nome_funcao FROM Funcao NATURAL JOIN Detalhamento_Funcao "+
                      "WHERE cod_subfuncao IN (SELECT cod_subfuncao FROM (SELECT cod_subfuncao, MAX(NRO_FUNCAO) "+
                          "FROM (SELECT cod_subfuncao, COUNT(cod_funcao) NRO_FUNCAO FROM "+
                              "Funcao NATURAL JOIN Detalhamento_Funcao GROUP BY cod_subfuncao)))", conn)
df

# 10. Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor), ordenadas de forma decrescente pelo valor, em que o programa de governo é aquele que possui o maior valor total em despesas durante o ano de 2018.

In [ ]:
df= pd.read_sql_query("SELECT cod_despesa, mes_despesa, ano_despesa, modalidade_despesa, valor_despesa FROM Despesa_Publica "+
                      "WHERE ano_despesa=2018 AND cod_programa_governo = "+
                          "(SELECT cod_programa_governo FROM Despesa_Publica WHERE valor_despesa="+
                              "(SELECT MAX(valor_despesa) FROM Despesa_Publica))"+
                      " ORDER BY valor_despesa DESC", conn)
df

# 9. Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor) em que o órgão responsável é aquele que possui um número de órgãos subordinados maior do que a média.

# 7. Liste os programas de governo e a quantidade de órgãos governamentais distintos envolvidos nas despesas públicas desses programas. Nesta consulta, desconsidere valores de despesa negativos ou iguais a zero no período.

In [ ]:
df= pd.read_sql_query("SELECT nome_programa_governo, count(cod_orgao) as NUMERO_ORGAOS, valor_despesa "+
                      "FROM Orgao NATURAL JOIN Despesa_Publica NATURAL JOIN Programa_Governo " +
                      "WHERE valor_despesa>0 GROUP BY cod_programa_governo "+
                      "ORDER BY nome_programa_governo", conn)
df

# 11. Liste os códigos e nomes dos órgãos governamentais relacionados ao programa de governo (ou programas, caso haja empate) que alcançou mais registros de despesas públicas durante o ano.


In [ ]:
df= pd.read_sql_query("SELECT cod_orgao, nome_orgao FROM Orgao NATURAL JOIN Despesa_Publica WHERE cod_programa_governo IN "+
                          "(SELECT cod_programa_governo FROM (SELECT cod_programa_governo, max(NRO_REGISTROS) FROM "+
                              "(SELECT cod_programa_governo, count(cod_programa_governo) as NRO_REGISTROS "+
                                  "FROM Despesa_Publica GROUP BY cod_programa_governo)))",conn)
df

In [ ]:
#AUXILIAR PARA 11
df= pd.read_sql_query("SELECT cod_programa_governo, count(cod_programa_governo) as NRO_REGISTROS FROM Despesa_Publica "+
                                  "GROUP BY cod_programa_governo",conn)
df

In [ ]:
#AUXILIAR DA AUXILIAR
df= pd.read_sql_query("SELECT sum(NRO_REGISTROS) FROM (SELECT count(cod_programa_governo) as NRO_REGISTROS FROM Despesa_Publica "+
                                  "GROUP BY cod_programa_governo)",conn)
df

# 2. Liste a média de despesas públicas por função (nome) e subfunção (nome), apenas nos casos em que a média excedeu mil reais.

In [ ]:
df= pd.read_sql_query("SELECT nome_funcao, nome_subfuncao, avg(valor_despesa) AS MEDIA_DESPESAS "+
                      "FROM (Subfuncao NATURAL JOIN Detalhamento_Funcao NATURAL JOIN Funcao) "+
                      "NATURAL JOIN Despesa_Publica GROUP BY cod_funcao, cod_subfuncao HAVING avg(valor_despesa)>1000 "+
                      "ORDER BY avg(valor_despesa) DESC", conn)
df

In [ ]:
#(8+1+1+1+1)Liste a media de despesas publicas por func~ao (nome) e subfunc~ao (nome), apenas nos
#casos em que a media excedeu mil reais.
df= pd.read_sql_query("SELECT nome_funcao, nome_subfuncao, avg(valor_despesa) AS MEDIA_DESPESAS "+
                      "FROM (Subfuncao NATURAL JOIN Detalhamento_Funcao NATURAL JOIN Funcao) "+
                      "NATURAL JOIN Despesa_Publica GROUP BY cod_despesa HAVING avg(valor_despesa)>1000 "+
                      "ORDER BY avg(valor_despesa) DESC", conn)
df

In [ ]:
df= pd.read_sql_query("SELECT DISTINCT nome_funcao, valor_despesa "+
                      "FROM Funcao NATURAL JOIN Detalhamento_Funcao NATURAL JOIN Despesa_Publica", conn)
df

In [ ]:
df= pd.read_sql_query("SELECT DISTINCT modalidade_despesa, valor_despesa, nome_orgao "+
                      "FROM Orgao NATURAL JOIN (SELECT * FROM Despesa_Publica "+
                      "WHERE modalidade_despesa= 'Reserva de Contingência')", conn)
df

In [ ]:
df= pd.read_sql_query("SELECT DISTINCT valor_despesa, nome_programa_governo "+
                      "FROM Despesa_Publica NATURAL JOIN (SELECT * FROM Programa_Governo "+
                      "WHERE nome_programa_governo='ENEM')", conn)
df

In [ ]:
df= pd.read_sql_query("SELECT DISTINCT nome_programa_governo, qtd_orgaos_subordinados, valor_despesa "+
                      "FROM Despesa_Publica NATURAL JOIN (SELECT * FROM Orgao "+
                      "WHERE nome_orgao= 'Ministério da Educação') NATURAL JOIN Programa_Governo", conn)
df

In [ ]:
conn.close()